In [22]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from keras.layers import Conv1D, MaxPooling1D
import tensorflow as tf

In [23]:
version = "cnn_v1"

model_path = f"../models/model_{version}.weights.h5"
tokenizer_path = f"../models/tokenizer_{version}.pkl"
class_path = f"../models/disease_classes_{version}.txt"

In [24]:
## Creating the model_loaded based on training model_loaded architecture
sent_length=20
voc_size = 50000
embedding_vector_features=40
model_loaded = None
num_classes = 30

# # Define CNN-LSTM model_loaded
model_loaded = Sequential()
model_loaded.add(Embedding(voc_size,embedding_vector_features,input_shape=(sent_length,)))
model_loaded.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_loaded.add(MaxPooling1D(pool_size=2))
model_loaded.add(LSTM(100))
model_loaded.add(Dropout(0.3))
model_loaded.add(Dense(num_classes,activation='softmax'))
model_loaded.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


print(model_loaded.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 20, 40)              │       2,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 18, 64)              │           7,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 9, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 100)                 │          66,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 30)                  │           3,030 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,076,774 (7.92 MB)

 Trainable params: 2,076,774 (7.92 MB)

 Non-trainable params: 0 (0.00 B)

None


In [25]:
# loading saved model weights
model_loaded.load_weights(model_path)

In [26]:
# Preprocessing function
import nltk
import re
import pickle
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from word_process import WordProcess

nltk.download('stopwords')

tokenizer = pickle.load(open(tokenizer_path, 'rb'))
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
wp = WordProcess()



def preprocess_text(text):
    """Preprocesses a single text sample for disease prediction."""
    # voc_size = 5000
    sent_length = 20
    processed_text = wp.process_sent2sent(text)

    # One-hot encoding and padding
    # print(processed_text)
    onehot_vector = tokenizer.texts_to_sequences([processed_text])
    # print('vector',onehot_vector)
    padded_vector = pad_sequences(onehot_vector, padding='pre', maxlen=sent_length)

    return padded_vector[0].tolist()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aller\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aller\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aller\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\aller\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:
import numpy as np

In [28]:
import json 

with open(class_path) as f:
    disease_classes = json.load(f)
disease_classes, len(disease_classes)

(['acne',
  'allergy',
  'arthritis',
  'bronchial asthma',
  'cervical spondylosis',
  'chicken pox',
  'common cold',
  'dengue',
  'diabetes',
  'dimorphic hemorrhoids',
  'drug reaction',
  'flu',
  'fungal infection',
  'gastroesophageal reflux disease',
  'hiv/aids',
  'hypertension',
  'impetigo',
  'irritable bowel syndrome',
  'jaundice',
  'malaria',
  'migraine',
  'peptic ulcer disease',
  'pneumonia',
  'psoriasis',
  'strep throat',
  'typhoid',
  'urinary tract infection',
  'varicose veins',
  'vitamin d deficiency',
  'yeast infection'],
 30)

In [29]:
test_cases = [
"I have been sneezing frequently, accompanied by a mild headache, runny nose, and a general feeling of being unwell.",
"Experiencing a low-grade fever with chills, nasal congestion, and a scratchy throat.",
"Mild body aches with a runny nose, a few sneezes, and feeling slightly fatigued.",
"Congested nose with a sore throat, slight cough, and sneezing fits.",
"I am experiencing itching and irritation in the vaginal area, along with a white, clumpy discharge that resembles cottage cheese.",
"There's a burning sensation during urination and redness and swelling of the vulva.",
"Feeling soreness and experiencing painful intercourse, accompanied by a thick, odorless, white vaginal discharge.",
"Persistent itching and a thick white discharge, with slight redness around the external genitalia.",
"Feeling tired all the time and my bones ache, especially in the joints and back. There's also muscle weakness.",
"Noticing more hair falling out, general fatigue, and aching bones. I've been indoors most of the time.",
"Experiencing bone pain and muscle weakness, feeling depressed more frequently.",
"My doctor mentioned bone softening, and I feel persistently low energy and down in mood.",
"My stomach cramps after eating and I frequently have diarrhea or constipation, feeling bloated.",
"Experiencing abdominal pain, bloating, and an inconsistent stool pattern, swinging between diarrhea and constipation.",
"Frequent bloating and gas with episodes of constipation followed by sudden diarrhea.",
"Abdominal discomfort, altered bowel habits, with bouts of diarrhea and periods of constipation, including bloating."
]

In [30]:
for test in test_cases:
    print(test)
    ind = model_loaded.predict(np.array([preprocess_text(test)]),verbose=0).argmax()
    print( disease_classes[ind])

I have been sneezing frequently, accompanied by a mild headache, runny nose, and a general feeling of being unwell.
allergy
Experiencing a low-grade fever with chills, nasal congestion, and a scratchy throat.
flu
Mild body aches with a runny nose, a few sneezes, and feeling slightly fatigued.
flu
Congested nose with a sore throat, slight cough, and sneezing fits.
allergy
I am experiencing itching and irritation in the vaginal area, along with a white, clumpy discharge that resembles cottage cheese.
yeast infection
There's a burning sensation during urination and redness and swelling of the vulva.
migraine
Feeling soreness and experiencing painful intercourse, accompanied by a thick, odorless, white vaginal discharge.
yeast infection
Persistent itching and a thick white discharge, with slight redness around the external genitalia.
yeast infection
Feeling tired all the time and my bones ache, especially in the joints and back. There's also muscle weakness.
vitamin d deficiency
Noticing m